# Dictionary operations

In [1]:
dict1_template = {
    'a': 1,
    'b': 2,
    'c': 3,
}
dict2_template = {
    'a': 11,
    'b': 12,
    'c': 13,
    'd': 14,
    'e': 15,
}

In [2]:
def copy_dicts() -> tuple[dict, dict]:
    return dict1_template.copy(), dict2_template.copy()

## Values

In [3]:
dict1, dict2 = copy_dicts()

In [4]:
# .values() returns iterable (not iterator, can be iterated multiple times)
dict1.values()

dict_values([1, 2, 3])

In [5]:
# reversible
reversed(dict1.values())

In [6]:
# original dict mapped through `mapping` attribute
dict1.values().mapping

mappingproxy({'a': 1, 'b': 2, 'c': 3})

In [7]:
# sorted() returns a list
sorted(dict1.values())

[1, 2, 3]

In [8]:
# mazání položky v dict
print(dict1)
try:
    del dict1['a']
except KeyError:
    print('Key a is not present in the dictionary.')
print(dict1)

{'a': 1, 'b': 2, 'c': 3}
{'b': 2, 'c': 3}


In [9]:
# not subscriptable!
dict1.values()[0]

TypeError: 'dict_values' object is not subscriptable

## Union / joining / chaining
* https://betterprogramming.pub/new-union-operators-to-merge-dictionaries-in-python-3-9-8c7dbbd1080c

### Older solutions for union

In [10]:
dict1, dict2 = copy_dicts()

In [11]:
{**dict1,**dict2}

{'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15}

In [12]:
dict(dict1,**dict2)

{'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15}

In [13]:
# in-place
dict1.update(dict2)
dict1

{'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15}

### Union operator since Python 3.9

In [ ]:
dict1, dict2 = copy_dicts()

In [ ]:
# like {**dict1,**dict2}
dict1 | dict2

In [ ]:
dict2 | dict1

In [ ]:
# like dict1.update(dict2)
dict1 |= dict2
dict1

### collections.ChainMap
* https://docs.python.org/3/library/collections.html#collections.ChainMap
* Access multiple dictionaries through a mapping.
* read / write
* Lookups search the underlying mappings successively until a key is found.
* Writes, updates, and deletions only operate on the first mapping.

In [ ]:
dict1, dict2 = copy_dicts()

In [18]:
import collections

cm = collections.ChainMap(dict1, dict2)
cm

ChainMap({'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15}, {'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15})

In [ ]:
for key in cm:
    print(f'{key}: {cm[key]}')

In [ ]:
cm['b'] = 5
dict(cm)

In [ ]:
# Setting a key present only in other dictionaries than the first one still creates it in the first dictionary
cm['d'] = 214
cm

In [ ]:
# Keys can be deleted from the first mapped dictionary
del cm['b']
dict(cm)

In [ ]:
# Not from the other ones
del cm['e']
dict(cm)

In [36]:
# ChainMap of empty dictionaries translates to False
cm = collections.ChainMap({}, {}, {})
bool(cm)

False

### Read-only ChainMap

In [ ]:
dict1, dict2 = copy_dicts()

In [16]:
import types

In [31]:
cm = collections.ChainMap(types.MappingProxyType({}), dict1, dict2)
cm

ChainMap(mappingproxy({}), {'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15}, {'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15})

In [24]:
cm2 = types.MappingProxyType(collections.ChainMap(dict1, dict2))
cm2

mappingproxy({'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15})

In [23]:
dict(cm)

{'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15}

In [20]:
del cm['a']

TypeError: 'mappingproxy' object does not support item deletion

In [21]:
cm['a'] = 555

TypeError: 'mappingproxy' object does not support item assignment

In [22]:
cm['zzz'] = 1

TypeError: 'mappingproxy' object does not support item assignment

### ChainMap reimplementation

In [50]:
# https://code.activestate.com/recipes/305268/

import UserDict

class Chainmap(UserDict.DictMixin):
    """Combine multiple mappings for sequential lookup.

    For example, to emulate Python's normal lookup sequence:

        import __builtin__
        pylookup = Chainmap(locals(), globals(), vars(__builtin__))        
    """

    def __init__(self, *maps):
        self._maps = maps

    def __getitem__(self, key):
        for mapping in self._maps:
            try:
                return mapping[key]
            except KeyError:
                pass
        raise KeyError(key)

if __name__ == "__main__":
    d1 = {'a':1, 'b':2}
    d2 = {'a':3, 'd':4}
    cm = Chainmap(d1, d2)
    assert cm['a'] == 1
    assert cm['b'] == 2
    assert cm['d'] == 4
    try:
        print cm['f']
    except KeyError:
        pass
    else:
        raise Exception('Did not raise KeyError for missing key')
    assert 'a' in cm  and  'b' in cm  and  'd' in cm
    assert cm.get('a', 10) == 1
    assert cm.get('b', 20) == 2
    assert cm.get('d', 30) == 4
    assert cm.get('f', 40) == 40

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2950139134.py, line 33)

### ChainMap iteration order
Dictionariees in ChainMap are iterated in reverse order, items in them in normal order

In [49]:
dict1 = {'a': 1, 'b': 2}
dict2 = {'c': 3, 'd': 4}
dict3 = {'e': 5, 'f': 6}

from collections import ChainMap

cm = ChainMap(dict1, dict2, dict3)
print(dict(cm))
cm1 = ChainMap(*reversed(cm.maps))   # we can create a new map with reversed maps
print(dict(cm1))

cm.maps.reverse()                    # or we can reverse list of maps in-place
print(dict(cm))

{'e': 5, 'f': 6, 'c': 3, 'd': 4, 'a': 1, 'b': 2}
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
